In [1]:
import csv
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, log_loss
import seaborn as sns
import os
import ast
import sys
import re
from statistics import mean, stdev
statistics_path = os.path.abspath("../")
sys.path.append(statistics_path)
import stats_utils
from matplotlib.ticker import MaxNLocator

In [2]:
path_to_exp_statistics = "/Users/admin/Desktop/thesis/dataset/metrics/"
path_to_exp_images = "/Users/admin/Desktop/thesis_writing/experiment_images/energy_experiments"

### We will compare the energy consumed on these circumstanes

1. model := [16, 5, 5, 6]
2. model := [16, 10, 10, 6]
3. model := [16, 16, 16, 6]
4. model := [16, 32, 32, 6]
5. model := [16, 64, 64, 6]

In [3]:
exp_name = "exp_7_1_1"
df1 = stats_utils.parse_experiments_statistics_to_df(path_to_exp_statistics, exp_name, csv_filename="logs.csv")

exp_name = "exp_7_2_1"
df2 = stats_utils.parse_experiments_statistics_to_df(path_to_exp_statistics, exp_name, csv_filename="logs.csv")

exp_name = "exp_7_3_1"
df3 = stats_utils.parse_experiments_statistics_to_df(path_to_exp_statistics, exp_name, csv_filename="logs.csv")

exp_name = "exp_7_4_1"
df4 = stats_utils.parse_experiments_statistics_to_df(path_to_exp_statistics, exp_name, csv_filename="logs.csv")

exp_name = "exp_7_5_1"
df5 = stats_utils.parse_experiments_statistics_to_df(path_to_exp_statistics, exp_name, csv_filename="logs.csv")

dfs = [df1, df2, df3, df4, df5]

In [6]:
label_names = [
    "[16, 5, 5, 6]",
    "[16, 10, 10, 6]",
    "[16, 16, 16, 6]",
    "[16, 32, 32, 6]",
    "[16, 64, 64, 6]"
]

In [4]:
def get_total_charge_drop(df):
    energy_df = stats_utils.extract_energy_data(df)
    charge_drops = energy_df
    charge_drops["charge_drop"] = charge_drops["charge_before"] - charge_drops["charge_after"]

    total_charge_drop_dict = {}
    # Plot each device's charge drop separately and add cumulative information to the legend
    for device in energy_df["device"].unique():
        subset = charge_drops[charge_drops["device"] == device]

        # Calculate cumulative charge consumption (first charge_before and last charge_after)
        first_charge_before = subset["charge_before"].iloc[0] / 1000
        last_charge_after = subset["charge_after"].iloc[-1] / 1000
        total_charge_drop = first_charge_before - last_charge_after
        
        total_charge_drop_dict[device] = total_charge_drop

    return total_charge_drop_dict

In [51]:
should_save = True
filename = "epoch_vs_federated_rounds"

final_accuracies = []
for df in dfs:
    first_client_name = ast.literal_eval(df['devices_names'][0])[0]
    acc, _ = stats_utils.get_accuracy_loss_values_for_dfs(df, first_client_name)
    final_accuracies.append(acc[-1])

# Collect charge drop per device per experiment
device_names = set()
all_charge_drops = []

for df in dfs:
    drop_dict = get_total_charge_drop(df)
    device_names.update(drop_dict.keys())
    all_charge_drops.append(drop_dict)

device_names.remove('samsung SM-A510F')
device_names.remove('Xiaomi M2006C3MNG')

device_names = sorted(device_names)
n_labels = len(label_names)
n_devices = len(device_names)

# Build matrix: rows = training settings, cols = devices
charge_matrix = np.full((n_labels, n_devices), np.nan)
for i, drop_dict in enumerate(all_charge_drops):
    for j, device in enumerate(device_names):
        value = drop_dict.get(device)
        if value is not None and not pd.isna(value):
            charge_matrix[i, j] = value

avg_drops = np.nanmean(charge_matrix, axis=1)
V = 3.7

# Plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), gridspec_kw={'height_ratios': [2, 1]})

# 1. Per-device charge bar plot
width = 0.12
x = np.arange(n_labels)

for i, device in enumerate(device_names):
    offset = (i - n_devices / 2) * width + width / 2
    ax1.bar(x + offset, charge_matrix[:, i] * V, width=width, label=device)

ax1.set_title("Energy Consumed per Device Across Training Configurations")
ax1.set_xticks(x)
ax1.set_xticklabels(label_names, rotation=20)
ax1.set_ylabel("Energy Consumption (mWh)")
ax1.legend(title="Device", bbox_to_anchor=(1.01, 1), loc="upper left")
ax1.grid(True, axis='y')

# 2. Avg charge + accuracy
bar = ax2.bar(x, final_accuracies, width=0.4, label="Final Accuracy", color="skyblue")
ax2.set_ylabel("Final Accuracy")
ax2.set_ylim(0, 1)
ax2.set_xticks(x)
ax2.set_xticklabels(label_names, rotation=20)
ax2.grid(True, axis='y')

ax3 = ax2.twinx()
total_energy_consumed = np.sum(charge_matrix, axis=1) * V + (2 * V * avg_drops)
ax3.plot(x,  total_energy_consumed, color="red", marker="o", label="Total Energy Consumption (mWh)")
#ax3.set_ylabel("Avg. Charge Drop (mAh)")
ax3.set_ylabel("Total Energy Consumption (mWh)")
ax3.set_ylim(0, np.nanmax(total_energy_consumed) * 1.1)

# Shared legend
lines_labels = [ax2.get_legend_handles_labels(), ax3.get_legend_handles_labels()]
lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
fig.legend(lines, labels, loc="upper center", ncol=2, bbox_to_anchor=(0.5, 0.99))

# fig.suptitle("Epoch vs Federated Round Trade-off: Device-Level and Average Energy Consumption")
plt.tight_layout(rect=[0, 0, 1, 0.96])

if not should_save:
    plt.show()
else:
    path_to_file = os.path.join(path_to_exp_images, filename + ".png")
    fig.savefig(path_to_file, dpi=300)
    plt.close(fig)

In [52]:
temp_df = stats_utils.extract_energy_data(df1)

In [53]:
temp_df.head()

,n_round,device,temperature_before,temperature_after,current_now_before,current_now_after,current_avg_before,current_avg_after,capacity_before,capacity_after,voltage_before,voltage_after,charge_before,charge_after
0,1,samsung SM-J730F,24.6,24.6,-1.130000e+02,-2.140000e+02,-1.180000e+02,-1.270000e+02,85,85,4034.0,4034.0,3060000.0,3060000.0
1,1,samsung SM-G975F,27.0,27.0,-1.690000e+02,-8.410000e+02,-1.490000e+02,-1.900000e+02,87,87,4038.0,4038.0,3451376.0,3451376.0
2,1,samsung SM-A510F,25.1,25.1,-1.930000e+02,-1.930000e+02,-1.210000e+02,-1.210000e+02,94,94,4076.0,4076.0,NaN,NaN
3,1,realme RMX3370,26.3,26.3,9.600000e+01,2.610000e+02,-2.147484e+09,-2.147484e+09,19,19,3639.0,3639.0,720000.0,720000.0
4,1,Xiaomi M2006C3MNG,25.8,25.8,-2.147484e+09,-2.147484e+09,-1.920000e+05,-1.920000e+05,82,82,4095.0,4095.0,4158220.0,4158220.0


In [54]:
total_energy_consumed

array([1120.138     , 1073.27133333,  977.58933333, 1053.279     ,
       1073.27133333])